# Workflow Construction Verification
This notebook attempts to implement the workflow construction proposal (`docs/dev_notes/workflow_construction_proposal.py`) using the current `fancy_core` library implementation to identify discrepancies and failures.

In [ ]:
import sys
import os
import inspect

# Add src to path so we can import fancy_core
current_dir = os.getcwd()
src_path = os.path.join(current_dir, 'src')
if src_path not in sys.path:
    sys.path.append(src_path)
    print(f"Added {src_path} to sys.path")

# 1. Attempt Import of Core Components
print("--- Importing Core Components ---")
try:
    from fancy_core.workflow import Workflow
    from fancy_core.decorators import step
    from fancy_core.cells import FancyCell
    from fancy_core.workflow_step import WorkflowStep
    print("SUCCESS: Imported Workflow, step, FancyCell, WorkflowStep")
except ImportError as e:
    print(f"FAILURE: Could not import core components: {e}")

In [ ]:
# 2. Check for Proposal-Specific Decorators
# The proposal uses @vectorize, @expand, @summarize. 
# We check if they exist in the library.

print("--- Checking for Advanced Decorators ---")
try:
    from fancy_core.decorators import vectorize, expand, summarize
    print("SUCCESS: Found @vectorize, @expand, @summarize")
except ImportError:
    print("FAILURE: @vectorize, @expand, @summarize are MISSING in src/fancy_core/decorators.py")
    print("Action: Creating mocks (aliased to @step) to continue testing the Workflow Context logic.")
    
    # Mocking them assuming they are specialized versions of @step for now
    def vectorize(func):
        return step(func)
    
    def expand(func):
        return step(func)
    
    def summarize(func):
        return step(func)


In [ ]:
# 3. Define Workflow Functions
# Using the mocked/imported decorators to define the YouTube analysis functions.

@expand
def fetch_channel_videos(url: str) -> list:
    """1 -> N: Gets a list of video URLs"""
    return ["video1", "video2"]

@vectorize
def extract_metadata(video_url: str) -> dict:
    """1 -> 1: Gets metadata for a video"""
    return {"title": "Test"}

@vectorize
def transcribe_video(video_url: str) -> str:
    """1 -> 1: Transcribes a video"""
    return "Transcript content"

@expand 
def segment_conversations(transcript: str, metadata: dict) -> list:
    """1 -> N: Cuts transcript into segments"""
    return [{"text": "segment"}]

@vectorize
def analyze_sentiment(conversation: dict) -> float:
    """1 -> 1: Analyzes sentiment"""
    return 0.9

@summarize
def generate_report(sentiments: list) -> str:
    """N -> 1: Aggregates results"""
    return "Final Report"

print("Functions defined successfully.")

## Workflow Context Test
The core feature of the proposal is that calling these methods inside a `with Workflow(...)` block should automatically record steps to that workflow.

In [ ]:
# 4. Attempt to Build the Workflow
print("--- Starting Workflow Construction ---")

wf = Workflow(name="YouTube Analysis")

try:
    with wf:
        print(f"Context Active: {Workflow.get_current() == wf}")
        
        # Define Inputs
        channel_url = "https://youtube.com/@example"
        
        # Step 1: Fetch Videos
        # In current impl, non-Cell inputs are treated as config.
        videos = fetch_channel_videos(channel_url)
        print(f"Called 'fetch_channel_videos'. result type: {type(videos)}")
        # Expectation: result should be usable as input for next step
        
        # Step 2: Meta & Transcripts
        # 'videos' is a StepWiring object. @step handles StepWiring as input (Scenario B).
        meta = extract_metadata(videos)
        transcripts = transcribe_video(videos)
        print(f"Called 'extract_metadata' & 'transcribe_video'.")

        # Step 3: Segment
        conversations = segment_conversations(transcripts, meta)
        
        # Step 4: Sentiment
        sentiments = analyze_sentiment(conversations)
        
        # Step 5: Report
        final_report = generate_report(sentiments)
        
        print("Construction block finished.")

except Exception as e:
    print(f"CRITICAL ERROR during construction: {e}")
    # Print stack trace if needed, but error message is usually enough for high level check

In [ ]:
# 5. Verify Workflow State
print("--- Verifying Workflow Content ---")
print(f"Workflow Name: {wf.name}")
print(f"Step Count: {len(wf.steps)}")

if len(wf.steps) == 0:
    print("\n[CONCLUSION] FAILURE:")
    print("The Workflow object is empty.")
    print("Reason: The @step decorator in 'src/fancy_core/decorators.py' creates a StepWiring object but") 
    print("DOES NOT add the step to 'Workflow.get_current()'.")
    print("The automatic context-capture mechanism described in the proposal is not implemented.")
else:
    print("\n[CONCLUSION] SUCCESS: Steps were recorded automatically!")
    for s in wf.steps:
        print(f" - {s.function_slug}")